# All analysis code for B. Danskin et al. 2023
Clears and reloads between each analysis type

In [1]:
%load_ext autoreload

# Behavior analysis

## General import functions

In [2]:
%reset -f
import sys
from os.path import dirname, join as pjoin
from os import listdir
sys.path.append('C:\\jupyter_notebooks\\Danskin_SciAdv_2023\\py_code') # set local directory
import numpy as np
import pickle
import random
import xarray as xr
import pandas as pd

import h5py
import scipy.io as sio
from scipy import stats
import statsmodels as sm # import statsmodels.api as sm
from scipy.optimize import minimize, basinhopping, curve_fit
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm, trange

In [3]:
## Directories
%autoreload
import bdanskin as BD

project_dir = 'C:\\jupyter_notebooks\\Danskin_SciAdv_2023' # local directory
behavior_dir = pjoin(project_dir, 'hattori_datasets_behavior')

## Logistic regresssion analysis

### Functions

#### pandas apply fit logistic

In [32]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression

In [23]:
def modelfit_logistic_pd(row, reg_type):
    n_back  = 10
    a = row['a']
    R = row['R']
    n_choice = sum((a==1)|(a==2))
    
    # Fit RUC
    glm_mat, glm_target = BD.logistic_prepare_predictors(a,R,n_back,'RUC')
    if reg_type=='none':
        log_reg = LogisticRegression(solver='newton-cg', penalty='none', n_jobs=-1, random_state=0)
    else: 
        log_reg = LogisticRegressionCV(solver='saga', cv=5, penalty=reg_type, n_jobs=-1, random_state=0, refit=True)
    fit_mdl = log_reg.fit(glm_mat,glm_target)  
    coef = fit_mdl.coef_[0]
    intercept = fit_mdl.intercept_[0]
    loglik,AIC,p_logit = BD.logistic_calc_loglik(glm_mat, glm_target,coef, intercept, n_back)
    raw_weights = np.append(coef,intercept)
    
    row['mdl_hist'] = n_back
    row['mdl_penalty'] = reg_type
    row['RUC_weights'] = raw_weights
    row['RUC_AIC'] = AIC
    row['RUC_loglik'] = loglik
    row['RUC_norm_loglik'] = loglik/n_choice
    
    # Fit RC
    glm_mat, glm_target = BD.logistic_prepare_predictors(a,R,n_back,'RC')
    if reg_type=='none':
        log_reg = LogisticRegression(solver='newton-cg', penalty='none', n_jobs=-1, random_state=0)
    else: 
        log_reg = LogisticRegressionCV(solver='saga', cv=5, penalty=reg_type, n_jobs=-1, random_state=0, refit=True)
    fit_mdl = log_reg.fit(glm_mat,glm_target)  
    coef = fit_mdl.coef_[0]
    intercept = fit_mdl.intercept_[0]
    loglik,AIC,p_logit = BD.logistic_calc_loglik(glm_mat, glm_target,coef, intercept, n_back)
    raw_weights = np.append(coef,intercept)
    
    row['RC_weights'] = raw_weights
    row['RC_AIC'] = AIC
    row['RC_loglik'] = loglik
    row['RC_norm_loglik'] = loglik/n_choice
    
    # Fit RU
    glm_mat, glm_target = BD.logistic_prepare_predictors(a,R,n_back,'RU')
    if reg_type=='none':
        log_reg = LogisticRegression(solver='newton-cg', penalty='none', n_jobs=-1, random_state=0)
    else: 
        log_reg = LogisticRegressionCV(solver='saga', cv=5, penalty=reg_type, n_jobs=-1, random_state=0, refit=True)
    fit_mdl = log_reg.fit(glm_mat,glm_target)  
    coef = fit_mdl.coef_[0]
    intercept = fit_mdl.intercept_[0]
    loglik,AIC,p_logit = BD.logistic_calc_loglik(glm_mat, glm_target,coef, intercept, n_back)
    raw_weights = np.append(coef,intercept)
    
    row['RU_weights'] = raw_weights
    row['RU_AIC'] = AIC
    row['RU_loglik'] = loglik
    row['RU_norm_loglik'] = loglik/n_choice
    
    # Fit R
    glm_mat, glm_target = BD.logistic_prepare_predictors(a,R,n_back,'R')
    if reg_type=='none':
        log_reg = LogisticRegression(solver='newton-cg', penalty='none', n_jobs=-1, random_state=0)
    else: 
        log_reg = LogisticRegressionCV(solver='saga', cv=5, penalty=reg_type, n_jobs=-1, random_state=0, refit=True)
    fit_mdl = log_reg.fit(glm_mat,glm_target)  
    coef = fit_mdl.coef_[0]
    intercept = fit_mdl.intercept_[0]
    loglik,AIC,p_logit = BD.logistic_calc_loglik(glm_mat, glm_target,coef, intercept, n_back)
    raw_weights = np.append(coef,intercept)
    
    row['R_weights'] = raw_weights
    row['R_AIC'] = AIC
    row['R_loglik'] = loglik
    row['R_norm_loglik'] = loglik/n_choice

    return row

### Load Imaging and simulation datasets

In [24]:
# Load data
imaging_beh_df = pd.read_pickle(pjoin(behavior_dir,'all_behavior_df.pkl'))

# 10000 trial simulation
sim_beh_df = pd.read_pickle(pjoin(behavior_dir, 'rl_simulation_full_base_all_imaging_10000.pkl'))
sim_beh_df = sim_beh_df[['Area','Mouse','Date','RL_Model','a','R']]

full_sim = sim_beh_df.loc[sim_beh_df['RL_Model']=='rl_full'].reset_index(drop=True)
base_sim = sim_beh_df.loc[sim_beh_df['RL_Model']=='rl_base'].reset_index(drop=True)

### Fit logistic weights

In [33]:
tqdm.pandas()
# Fit imaging data
beh_df = imaging_beh_df.iloc[:2]
reg_df = beh_df.progress_apply(modelfit_logistic_pd,axis=1, args=['none'])
reg_df = reg_df.drop(columns=['a','R'])
reg_df.index.rename('session_id', inplace=True)
reg_df.to_pickle(pjoin(behavior_dir, 'logstic_imaging_10_hist.pkl'))

  0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
tqdm.pandas()
# Fit simulation data
beh_df = full_sim
reg_df = beh_df.progress_apply(modelfit_logistic_pd,axis=1, args=['none'])
reg_df = reg_df.drop(columns=['a','R'])
reg_df.index.rename('session_id', inplace=True)
reg_df.to_pickle(pjoin(behavior_dir, 'logstic_full_rl_10_hist.pkl'))

beh_df = base_sim
reg_df = beh_df.progress_apply(modelfit_logistic_pd,axis=1, args=['none'])
reg_df = reg_df.drop(columns=['a','R'])
reg_df.index.rename('session_id', inplace=True)
reg_df.to_pickle(pjoin(behavior_dir, 'logstic_base_rl_10_hist.pkl'))

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

## Behlogits

### Functions

#### pandas apply model fits

In [6]:
def modelfit_behlogit(row, n_back, mdl_type):
    rng = np.random.default_rng(220727)
    beh_func = eval('BD.logit_'+mdl_type)
    # Prepare history matrices
    a,R = row[['a','R']]
    a_hist,R_hist,uR_hist = BD.prepare_hist_matrix(a,R,n_back) # New code to simplify fits
    n_choice_trials = sum((a==1)|(a==2))
    
    # given different model types
    if (mdl_type=='exp_r') | (mdl_type=='hyp_r'):
        n_params = 3
        initial_params = [1,1,0]
        bounds = [[0, np.inf],[-np.inf,np.inf],[-np.inf, np.inf]]
    elif (mdl_type=='exp_ru') | (mdl_type=='hyp_ru') | (mdl_type=='hyp_r_exp_u') | (mdl_type=='exp_r_hyp_u'):
        n_params = 5
        initial_params = [1,1,-1,1,0]
        bounds = [[0, np.inf],[-np.inf,np.inf],[-np.inf, np.inf],[-np.inf, np.inf],[-np.inf, np.inf]]
    elif (mdl_type=='exp_rc') | (mdl_type=='hyp_rc') | (mdl_type=='hyp_r_exp_c'):
        n_params = 5
        initial_params = [1,1,-1,1,0]
        bounds = [[0, np.inf],[-np.inf,np.inf],[-np.inf, np.inf],[-np.inf, np.inf],[-np.inf, np.inf]]
    elif (mdl_type=='exp_ruc') | (mdl_type=='hyp_ruc')| (mdl_type=='hyp_r_exp_uc'):
        n_params = 7
        initial_params = [1,1,-1,1,1,1,0]
        bounds = [[0, np.inf],[-np.inf,np.inf],[-np.inf,0],[-np.inf,np.inf],
                  [0, np.inf],[-np.inf,np.inf],[-np.inf, np.inf]]
    else:
        print('unable to parse mdl_type')

    # Run k-fold cross validation
    cv_iter = 0
    loglik_vec = np.zeros(10)
    norm_loglik_vec = np.zeros(10)
    aic_vec = np.zeros(10)
    pa_vec = np.zeros(10)
    kf = KFold(n_splits=10, shuffle=False)#, random_state=0)
    kf.get_n_splits(np.arange(0,len(a)))
    for train_index, test_index in kf.split(np.arange(0,len(a))):
        lik_model = minimize(beh_func, initial_params, bounds=bounds, method='L-BFGS-B',
                         args=(a[train_index], a_hist[train_index,:], R_hist[train_index,:],
                               uR_hist[train_index,:], 1))
        temp_params = lik_model.x
        p_logit,nloglik = beh_func(temp_params, a[test_index],a_hist[test_index,:], 
                                   R_hist[test_index,:], uR_hist[test_index,:], 0)
        pa_vec[cv_iter] = BD.PA_logit(a[test_index],p_logit)
        aic_vec[cv_iter] = 2*n_params + 2*nloglik
        loglik_vec[cv_iter] = -nloglik
        norm_loglik_vec[cv_iter] = -nloglik/len(test_index)
        cv_iter+=1
        
    # Fit on full session
    minimizer_kwargs = {"method":"L-BFGS-B","bounds":bounds, 
                        "args":(a, a_hist, R_hist, uR_hist, 1)}
    with np.errstate(divide='ignore',invalid='ignore'):
        lik_model = basinhopping(beh_func, initial_params, minimizer_kwargs=minimizer_kwargs, niter=10)

    temp_params = lik_model.x
    p_logit,nloglik = beh_func(temp_params,a,a_hist,R_hist,uR_hist, 0)
    full_PA = BD.PA_logit(a,p_logit)
    full_AIC = 2*n_params + 2*nloglik 

    row['mdl_type'] = mdl_type
    row['mdl_hist'] = n_back
    row['full_session_AIC'] = full_AIC
    row['full_session_pa'] = full_PA
    row['full_session_loglik'] = -nloglik
    row['full_session_norm_loglik'] = -nloglik/n_choice_trials
    row['full_session_betaR'] = temp_params[0]
    row['full_session_tauR'] = np.exp(temp_params[1])

    if (mdl_type=='exp_r') | (mdl_type=='hyp_r'):
        row['full_session_betaU'] = np.nan
        row['full_session_tauU'] = np.nan
        row['full_session_betaC'] = np.nan
        row['full_session_tauC'] = np.nan
        row['full_session_const'] = temp_params[2]
    elif (mdl_type=='exp_ru') | (mdl_type=='hyp_ru') | (mdl_type=='hyp_r_exp_u') | (mdl_type=='exp_r_hyp_u'):
        row['full_session_betaU'] = temp_params[2]
        row['full_session_tauU'] = np.exp(temp_params[3])
        row['full_session_betaC'] = np.nan
        row['full_session_tauC'] = np.nan
        row['full_session_const'] = temp_params[4]
    elif (mdl_type=='exp_rc') | (mdl_type=='hyp_rc') | (mdl_type=='hyp_r_exp_c'):
        row['full_session_betaU'] = np.nan
        row['full_session_tauU'] = np.nan
        row['full_session_betaC'] = temp_params[2]
        row['full_session_tauC'] = np.exp(temp_params[3])
        row['full_session_const'] = temp_params[4]
    elif (mdl_type=='exp_ruc') | (mdl_type=='hyp_ruc') | (mdl_type=='hyp_r_exp_uc'):
        row['full_session_betaU'] = temp_params[2]
        row['full_session_tauU'] = np.exp(temp_params[3])
        row['full_session_betaC'] = temp_params[4]
        row['full_session_tauC'] = np.exp(temp_params[5])
        row['full_session_const'] = temp_params[6]

    row['CV_mean_loglik'] = np.nanmean(loglik_vec)
    row['CV_mean_norm_loglik'] = np.nanmean(norm_loglik_vec)
    row['CV_mean_aic'] = np.nanmean(aic_vec)
    row['CV_mean_pa'] = np.nanmean(pa_vec)
    
    return row

#### pandas apply constrained model fits

In [7]:
def modelfit_behlogit_constrained_ru(row, n_back, mdl_type):
    rng = np.random.default_rng(220727)
    beh_func = eval('BD.logit_'+mdl_type)
    # Prepare history matrices
    a,R = row[['a','R']]
    a_hist,R_hist,uR_hist = BD.prepare_hist_matrix(a,R,n_back) # New code to simplify fits
    n_choice_trials = sum((a==1)|(a==2))
    
    # given different model types
    if (mdl_type=='exp_ru') | (mdl_type=='hyp_ru') | (mdl_type=='hyp_r_exp_u') | (mdl_type=='exp_r_hyp_u'):
        n_params = 5
        initial_params = [1,1,-1,1,0]
        bounds = [[0, np.inf],[0,np.inf],[-np.inf, 0],[0, np.inf],[-np.inf, np.inf]]
    else:
        print('wrong model type provided')

    # Run k-fold cross validation
    cv_iter = 0
    loglik_vec = np.zeros(10)
    norm_loglik_vec = np.zeros(10)
    aic_vec = np.zeros(10)
    pa_vec = np.zeros(10)
    kf = KFold(n_splits=10, shuffle=False)#, random_state=0)
    kf.get_n_splits(np.arange(0,len(a)))
    for train_index, test_index in kf.split(np.arange(0,len(a))):
        lik_model = minimize(beh_func, initial_params, bounds=bounds, method='L-BFGS-B',
                         args=(a[train_index], a_hist[train_index,:], R_hist[train_index,:],
                               uR_hist[train_index,:], 1))
        temp_params = lik_model.x
        p_logit,nloglik = beh_func(temp_params, a[test_index],a_hist[test_index,:], 
                                   R_hist[test_index,:], uR_hist[test_index,:], 0)
        pa_vec[cv_iter] = BD.PA_logit(a[test_index],p_logit)
        aic_vec[cv_iter] = 2*n_params + 2*nloglik
        loglik_vec[cv_iter] = -nloglik
        norm_loglik_vec[cv_iter] = -nloglik/len(test_index)
        cv_iter+=1
        
    # Fit on full session
    minimizer_kwargs = {"method":"L-BFGS-B","bounds":bounds, 
                        "args":(a, a_hist, R_hist, uR_hist, 1)}
    with np.errstate(divide='ignore',invalid='ignore'):
        lik_model = basinhopping(beh_func, initial_params, minimizer_kwargs=minimizer_kwargs, niter=10)

    temp_params = lik_model.x
    p_logit,nloglik = beh_func(temp_params,a,a_hist,R_hist,uR_hist, 0)
    full_PA = BD.PA_logit(a,p_logit)
    full_AIC = 2*n_params + 2*nloglik 

    row['mdl_type'] = mdl_type
    row['mdl_hist'] = n_back
    row['full_session_AIC'] = full_AIC
    row['full_session_pa'] = full_PA
    row['full_session_loglik'] = -nloglik
    row['full_session_norm_loglik'] = -nloglik/n_choice_trials
    row['full_session_betaR'] = temp_params[0]
    row['full_session_tauR'] = np.exp(temp_params[1])

    if (mdl_type=='exp_r') | (mdl_type=='hyp_r'):
        row['full_session_betaU'] = np.nan
        row['full_session_tauU'] = np.nan
        row['full_session_betaC'] = np.nan
        row['full_session_tauC'] = np.nan
        row['full_session_const'] = temp_params[2]
    elif (mdl_type=='exp_ru') | (mdl_type=='hyp_ru') | (mdl_type=='hyp_r_exp_u') | (mdl_type=='exp_r_hyp_u'):
        row['full_session_betaU'] = temp_params[2]
        row['full_session_tauU'] = np.exp(temp_params[3])
        row['full_session_betaC'] = np.nan
        row['full_session_tauC'] = np.nan
        row['full_session_const'] = temp_params[4]
    elif (mdl_type=='exp_rc') | (mdl_type=='hyp_rc') | (mdl_type=='hyp_r_exp_c'):
        row['full_session_betaU'] = np.nan
        row['full_session_tauU'] = np.nan
        row['full_session_betaC'] = temp_params[2]
        row['full_session_tauC'] = np.exp(temp_params[3])
        row['full_session_const'] = temp_params[4]
    elif (mdl_type=='exp_ruc') | (mdl_type=='hyp_ruc') | (mdl_type=='hyp_r_exp_uc'):
        row['full_session_betaU'] = temp_params[2]
        row['full_session_tauU'] = np.exp(temp_params[3])
        row['full_session_betaC'] = temp_params[4]
        row['full_session_tauC'] = np.exp(temp_params[5])
        row['full_session_const'] = temp_params[6]

    row['CV_mean_loglik'] = np.nanmean(loglik_vec)
    row['CV_mean_norm_loglik'] = np.nanmean(norm_loglik_vec)
    row['CV_mean_aic'] = np.nanmean(aic_vec)
    row['CV_mean_pa'] = np.nanmean(pa_vec)
    
    return row

### Load Imaging and simulation datasets

In [8]:
# Load data
imaging_beh_df = pd.read_pickle(pjoin(behavior_dir,'all_behavior_df.pkl'))

# 10000 trial simulation
sim_beh_df = pd.read_pickle(pjoin(behavior_dir, 'rl_simulation_full_base_all_imaging_10000.pkl'))
sim_beh_df = sim_beh_df[['Area','Mouse','Date','RL_Model','a','R']]

full_sim = sim_beh_df.loc[sim_beh_df['RL_Model']=='rl_full'].reset_index(drop=True)
base_sim = sim_beh_df.loc[sim_beh_df['RL_Model']=='rl_base'].reset_index(drop=True)

### Fit Behlogit models (15-history)

In [37]:
tqdm.pandas()
# Fit imaging data
beh_df = imaging_beh_df#.iloc[:3]
print('exponential (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_r']))
behlogit_all = temp_df.drop(columns=['a','R'])
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_ru']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_r_hyp_u']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_rc']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

print('hyperbolic (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_ru']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_r_exp_u']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_rc']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

behlogit_all.to_pickle(pjoin(behavior_dir, 'behlogit_imaging_15_hist.pkl'))

exponential (15-hist)


  0%|          | 0/3 [00:00<?, ?it/s]

In [252]:
# Fit imaging data
beh_df = imaging_beh_df # .iloc[:10]
print('exponential (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit_constrained_ru,axis=1,args=([15,'exp_ru']))
behlogit_all = temp_df.drop(columns=['a','R'])
temp_df = beh_df.progress_apply(modelfit_behlogit_constrained_ru,axis=1,args=([15,'exp_r_hyp_u']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

print('hyperbolic (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit_constrained_ru,axis=1,args=([15,'hyp_ru']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit_constrained_ru,axis=1,args=([15,'hyp_r_exp_u']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

behlogit_all.to_pickle(pjoin(behavior_dir, 'behlogit_imaging_15_hist_constrained.pkl'))

exponential (15-hist)


  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

hyperbolic (15-hist)


  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [72]:
# Fit simulation data
beh_df = full_sim.copy() # .iloc[:10]
print('exponential (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_r']))
behlogit_all = temp_df.drop(columns=['a','R'])
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_ru']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_r_hyp_u']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_rc']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

print('hyperbolic (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_ru']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_r_exp_u']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_rc']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

behlogit_all.to_pickle(pjoin(behavior_dir, 'behlogit_rl_full_15_hist.pkl'))

beh_df = base_sim.copy()  # .iloc[:10]
print('exponential (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_r']))
behlogit_all = temp_df.drop(columns=['a','R'])
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_ru']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_r_hyp_u']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_rc']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

print('hyperbolic (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_ru']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_r_exp_u']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_rc']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

behlogit_all.to_pickle(pjoin(behavior_dir, 'behlogit_rl_base_15_hist.pkl'))

exponential (15-hist)


  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

...

hyperbolic (15-hist)


  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

### Fit Behlogit models (compare history)

In [11]:
tqdm.pandas()
# Fit imaging data
beh_df = imaging_beh_df#.iloc[:10]
print('exponential (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([5,'exp_r']))
behlogit_all = temp_df.drop(columns=['a','R'])
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([10,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([20,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([30,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

print('hyperbolic (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([5,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([10,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([20,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([30,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

behlogit_all.to_pickle(pjoin(behavior_dir, 'behlogit_imaging_compare_hist.pkl'))

exponential (15-hist)


  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

...

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [12]:
# Fit simulation data
beh_df = full_sim.copy() #.iloc[:10]
print('exponential (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([5, 'exp_r']))
behlogit_all = temp_df.drop(columns=['a','R'])
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([10,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([20,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([30,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

print('hyperbolic (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([5,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([10,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([20,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([30,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

behlogit_all.to_pickle(pjoin(behavior_dir, 'behlogit_rl_full_compare_hist.pkl'))


beh_df = base_sim.copy()  #.iloc[:10]
print('exponential (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([5, 'exp_r']))
behlogit_all = temp_df.drop(columns=['a','R'])
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([10,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([20,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([30,'exp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

print('hyperbolic (15-hist)')
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([5,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([10,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([15,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([20,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)
temp_df = beh_df.progress_apply(modelfit_behlogit,axis=1,args=([30,'hyp_r']))
behlogit_all = behlogit_all.append(temp_df.drop(columns=['a','R']), ignore_index=False)

behlogit_all.to_pickle(pjoin(behavior_dir, 'behlogit_rl_base_compare_hist.pkl'))

exponential (15-hist)


  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

...

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

# Imaging analysis

## General import functions

In [2]:
%reset -f
import sys
from os.path import dirname, join as pjoin
from os import listdir
sys.path.append('C:\\jupyter_notebooks\\Danskin_SciAdv_2023\\py_code') # set local directory
import numpy as np
import pickle
import random
import xarray as xr
import pandas as pd

import h5py
import scipy.io as sio
from scipy import stats
import statsmodels as sm # import statsmodels.api as sm
from scipy.optimize import minimize, basinhopping, curve_fit
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm, trange

In [3]:
## Directories
%autoreload
import bdanskin as BD 

project_dir = 'C:\\jupyter_notebooks\\Danskin_SciAdv_2023' # local directory
imaging_dir = pjoin(project_dir, 'hattori_datasets_xarray')
cellfits_dir = pjoin(project_dir, 'hattori_datasets_xarray_cellfits')
behavior_dir = pjoin(project_dir, 'hattori_datasets_behavior')

## CV model fit

### Functions

#### cross-validation function

In [50]:
def fit_cv(y, c0, a_hist_c, R_hist_c, uR_hist_c, mdl_dict):
    n_obs = len(y)  
    mdl_name = mdl_dict['mdl_name']
    fit_func = mdl_dict['fit_func']
    n_params = mdl_dict['n_params']
    n_pvals = mdl_dict['n_pvals']
    initial_params = mdl_dict['initial_params']
    bounds = mdl_dict['bounds']
    
    # Cross-validation
    n_fold = 10
    rsq_vec = np.zeros(n_fold)
    snr_vec = np.zeros(n_fold)
    aic_vec = np.zeros(n_fold)
    ll_vec = np.zeros(n_fold)
    norm_ll_vec = np.zeros(n_fold)
    param_mat = np.zeros([n_fold,n_params])
    pval_mat = np.zeros([n_fold, n_pvals])
    kf = KFold(n_splits=10, shuffle=False)#, random_state=0)
    kf.get_n_splits(np.arange(0,n_obs))
    cv_iter = 0
    for train_index, test_index in kf.split(np.arange(0,n_obs)):
        n_test = len(test_index)
        # Simple fit
        lik_model = minimize(fit_func, initial_params, bounds=bounds, method='L-BFGS-B',
                     args=(y[train_index], c0[train_index], a_hist_c[train_index,:],
                             R_hist_c[train_index,:], uR_hist_c[train_index,:], 1))
        temp_params = lik_model.x
        p_values = estimate_pval(temp_params, y[train_index], c0[train_index],
                                 a_hist_c[train_index,:],
                                 R_hist_c[train_index,:],
                                 uR_hist_c[train_index,:], mdl_dict)
        
        _,sse,rsq,snr = fit_func(temp_params, y[test_index], c0[test_index],
                                 a_hist_c[test_index,:],
                                 R_hist_c[test_index,:], 
                                 uR_hist_c[test_index,:], 0)
        loglik, norm_loglik = BD.calc_mdl_loglik(sse, n_test)
        aic = BD.calc_aic_loglik(loglik, n_params)
        
        rsq_vec[cv_iter] = rsq
        snr_vec[cv_iter] = snr
        aic_vec[cv_iter] = aic
        ll_vec[cv_iter] = loglik
        norm_ll_vec[cv_iter] = norm_loglik
        param_mat[cv_iter,:] = temp_params
        pval_mat[cv_iter] = p_values
        cv_iter+=1
    
    mean_params = np.nanmean(param_mat,0)
    std_params = np.nanstd(param_mat,0)
    mean_pvals = stats.gmean(pval_mat,0)
    out_dict = {'mean_params': mean_params, 'std_params': std_params, 
                'mean_pvals': mean_pvals, 'mean_rsq': np.nanmean(rsq_vec),
                'mean_snr': np.nanmean(snr_vec), 'mean_aic': np.nanmean(aic_vec),
                'mean_loglik': np.nanmean(ll_vec), 'mean_norm_loglik': np.nanmean(norm_ll_vec)}
    return out_dict

#### calculate p-values

In [44]:
def estimate_pval(temp_params, y, c0, a_hist_c, R_hist_c, uR_hist_c, mdl_dict):
    mdl_name = mdl_dict['mdl_name']
    y = y.values
    if (mdl_name=='null'):
        predictors = pd.DataFrame({'c0': c0})
    elif (mdl_name=='Rp1'):
        past_rc = np.multiply(a_hist_c, R_hist_c)
        rc_weights = past_rc[:,-1]
        predictors = pd.DataFrame({'RCp1': rc_weights,
                                       'c0': c0})
    elif (mdl_name=='exp_r'):
        rc_tau = np.exp(temp_params[1])
        rc_weights = BD.exp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        predictors = pd.DataFrame({'exp_rc': rc_weights,
                                       'c0': c0})
    elif (mdl_name=='hyp_r'):
        rc_tau = np.exp(temp_params[1])
        rc_weights = BD.hyp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        predictors = pd.DataFrame({'hyp_rc': rc_weights,
                                       'c0': c0})
    elif (mdl_name=='exp_ru') | (mdl_name=='exp_ru_con'):
        rc_tau = np.exp(temp_params[1])
        uc_tau = np.exp(temp_params[3])
        rc_weights = BD.exp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        uc_weights = BD.exp_filter(a_hist_c, R_hist_c, uc_tau,'ur')
        predictors = pd.DataFrame({'exp_rc': rc_weights, 'exp_uc': uc_weights,
                                   'c0': c0})
    elif (mdl_name=='hyp_ru') | (mdl_name=='hyp_ru_con'):
        rc_tau = np.exp(temp_params[1])
        uc_tau = np.exp(temp_params[3])
        rc_weights = BD.hyp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        uc_weights = BD.hyp_filter(a_hist_c, R_hist_c, uc_tau,'ur')
        predictors = pd.DataFrame({'hyp_rc': rc_weights, 'hyp_uc': uc_weights,
                                   'c0': c0})
    elif (mdl_name=='exp_r_hyp_u'):
        rc_tau = np.exp(temp_params[1])
        uc_tau = np.exp(temp_params[3])
        rc_weights = BD.exp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        uc_weights = BD.hyp_filter(a_hist_c, R_hist_c, uc_tau,'c')
        predictors = pd.DataFrame({'exp_rc': rc_weights, 'hyp_uc': uc_weights,
                                   'c0': c0})
    elif (mdl_name=='hyp_r_exp_u'):
        rc_tau = np.exp(temp_params[1])
        uc_tau = np.exp(temp_params[3])
        rc_weights = BD.hyp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        uc_weights = BD.exp_filter(a_hist_c, R_hist_c, uc_tau,'c')
        predictors = pd.DataFrame({'hyp_rc': rc_weights, 'exp_uc': uc_weights,
                                   'c0': c0})
       
    X2 = sm.add_constant(predictors) # Add constant term to predictor dataframe
    est = sm.OLS(y, X2).fit() # Fit linear regression with ordinary least squares
    p_values = est.pvalues
    return p_values

#### pandas apply cv fit

In [45]:
def apply_fit_CV_cell(column,c0,a_hist_c,R_hist_c,uR_hist_c,mdl_name):
    # mdl_name: 'exp_r' 'hyp_ruc' 'exp_r_con'
    rng = np.random.default_rng(220804)
    y = column
    n_obs = len(y)
    fit_func = eval('BD.reg_' + mdl_name + '_c0')
    
    if (mdl_name=='null'):
        n_params = 2; n_pvals = 2
        initial_params = [0.01, 0.01]
        bounds = [[-np.inf,np.inf],[-np.inf,np.inf]]
    elif (mdl_name=='Rp1'):
        n_params = 3; n_pvals = 3
        initial_params = [0.01, 0.01, 0.01]
        bounds = [[-np.inf,np.inf],[-np.inf,np.inf],[-np.inf,np.inf]]
    elif (mdl_name=='exp_r') | (mdl_name=='hyp_r'):
        n_params = 4; n_pvals = 3
        initial_params = [0.01, 1, 0.01, 0.01]
        bounds = [[-np.inf,np.inf],[-np.log(100),np.log(100)],[-np.inf,np.inf],[-np.inf,np.inf]]
    elif (mdl_name=='exp_ru') | (mdl_name=='hyp_ru'):
        n_params = 6; n_pvals = 4
        initial_params = [0.01, 1, -0.01, 1, 0.01, 0.01]
        bounds = [[-np.inf,np.inf],[-np.log(100),np.log(100)],[-np.inf,np.inf],[-np.log(100),np.log(100)],
                  [-np.inf,np.inf],[-np.inf,np.inf]]
    elif (mdl_name=='exp_r_hyp_u') | (mdl_name=='hyp_r_exp_u'):
        n_params = 6; n_pvals = 4
        initial_params = [0.01, 1, -0.01, 1, 0.01, 0.01]
        bounds = [[-np.inf,np.inf],[-np.log(100),np.log(100)],[-np.inf,0],[-np.log(100),np.log(100)],
                  [-np.inf,np.inf],[-np.inf,np.inf]]
    else:
        print('unable to parse mdl_type')
    
    # Fit with cross-validation
    mdl_dict = {'mdl_name': mdl_name, 'fit_func': fit_func, 'n_params': n_params, 'n_pvals': n_pvals,
                'initial_params': initial_params, 'bounds': bounds}
    cv_out_dict = fit_cv(y, c0, a_hist_c, R_hist_c, uR_hist_c, mdl_dict)
    temp_params = cv_out_dict['mean_params']
    std_params = cv_out_dict['std_params']
    p_values = cv_out_dict['mean_pvals']
    
    column['mdl_type'] = mdl_name
    column['cv_mean_rsq'] = cv_out_dict['mean_rsq']
    column['cv_mean_snr'] = cv_out_dict['mean_snr']
    column['cv_mean_aic'] = cv_out_dict['mean_aic']
    column['cv_mean_loglik'] = cv_out_dict['mean_loglik']
    column['cv_mean_norm_loglik'] = cv_out_dict['mean_norm_loglik']
    if (mdl_name=='null'):
        column['cv_mean_beta_RC'] =  np.nan
        column['cv_mean_tau_RC'] =  np.nan
        column['cv_mean_beta_UC'] = np.nan
        column['cv_mean_tau_UC'] = np.nan
        column['cv_mean_beta_C0'] = temp_params[0]
        column['cv_mean_beta_0'] = temp_params[1]
        column['cv_std_beta_RC'] =  np.nan
        column['cv_std_tau_RC'] =  np.nan
        column['cv_std_beta_UC'] =  np.nan
        column['cv_std_tau_UC'] =  np.nan
        column['cv_std_beta_C0'] =  std_params[0]
        column['cv_std_beta_0'] =  std_params[1]
        column['cv_gmean_p_beta_RC'] = np.nan
        column['cv_gmean_p_beta_UC'] = np.nan
        column['cv_gmean_p_beta_C0'] = p_values[1]
        column['cv_gmean_p_beta_0'] = p_values[0]
    elif (mdl_name=='Rp1'):
        column['cv_mean_beta_RC'] =  temp_params[0]
        column['cv_mean_tau_RC'] =  np.nan
        column['cv_mean_beta_UC'] = np.nan
        column['cv_mean_tau_UC'] = np.nan
        column['cv_mean_beta_C0'] = temp_params[1]
        column['cv_mean_beta_0'] = temp_params[2]
        column['cv_std_beta_RC'] =  std_params[0]
        column['cv_std_tau_RC'] =  np.nan
        column['cv_std_beta_UC'] =  np.nan
        column['cv_std_tau_UC'] =  np.nan
        column['cv_std_beta_C0'] =  std_params[1]
        column['cv_std_beta_0'] =  std_params[2]
        column['cv_gmean_p_beta_RC'] = p_values[1]
        column['cv_gmean_p_beta_UC'] = np.nan
        column['cv_gmean_p_beta_C0'] = p_values[2]
        column['cv_gmean_p_beta_0'] = p_values[0]
    elif (mdl_name=='exp_r') | (mdl_name=='hyp_r'):
        column['cv_mean_beta_RC'] =  temp_params[0]
        column['cv_mean_tau_RC'] =  np.exp(temp_params[1])
        column['cv_mean_beta_UC'] = np.nan
        column['cv_mean_tau_UC'] = np.nan
        column['cv_mean_beta_C0'] = temp_params[2]
        column['cv_mean_beta_0'] = temp_params[3]
        column['cv_std_beta_RC'] =  std_params[0]
        column['cv_std_tau_RC'] =  np.exp(std_params[1])
        column['cv_std_beta_UC'] =  np.nan
        column['cv_std_tau_UC'] =  np.nan
        column['cv_std_beta_C0'] =  std_params[2]
        column['cv_std_beta_0'] =  std_params[3]
        column['cv_gmean_p_beta_RC'] = p_values[1]
        column['cv_gmean_p_beta_UC'] = np.nan
        column['cv_gmean_p_beta_C0'] = p_values[2]
        column['cv_gmean_p_beta_0'] = p_values[0]
    elif (mdl_name=='exp_ru') | (mdl_name=='hyp_ru') | (mdl_name=='exp_r_hyp_u') | (mdl_name=='hyp_r_exp_u'):
        column['cv_mean_beta_RC'] =  temp_params[0]
        column['cv_mean_tau_RC'] =  np.exp(temp_params[1])
        column['cv_mean_beta_UC'] = temp_params[2]
        column['cv_mean_tau_UC'] = np.exp(temp_params[3])
        column['cv_mean_beta_C0'] = temp_params[4]
        column['cv_mean_beta_0'] = temp_params[5]
        column['cv_std_beta_RC'] =  std_params[0]
        column['cv_std_tau_RC'] =  np.exp(std_params[1])
        column['cv_std_beta_UC'] = std_params[2]
        column['cv_std_tau_UC'] = np.exp(std_params[3])
        column['cv_std_beta_C0'] = std_params[4]
        column['cv_std_beta_0'] = std_params[5]
        column['cv_gmean_p_beta_RC'] = p_values[1]
        column['cv_gmean_p_beta_UC'] = p_values[2]
        column['cv_gmean_p_beta_C0'] = p_values[3]
        column['cv_gmean_p_beta_0'] = p_values[0]
    return column

### Process Data

#### Estimate across sessions

In [51]:
sessions_all = BD.get_sessions('all')
for ss in trange(len(sessions_all)):
    mouse = sessions_all[ss][:5]
    date = sessions_all[ss][6:12]
    session = sessions_all[ss][6:]
    print('Checking analysis for {} on {}'.format(mouse, date))

    ds_2s = xr.open_dataset(pjoin(imaging_dir,'preprocess_data_2s','{}_ready_2s.nc'.format(sessions_all[ss])))

    # Prepare behavior information
    a = np.array(ds_2s.a)
    R = np.array(ds_2s.R)
    n_back = 15
    a_hist,R_hist,uR_hist = BD.prepare_hist_matrix(a,R,n_back)

    # Choice metrics
    choice_trials = (a==1)|(a==2)
    left_trials = (a==1)+0
    left_trials[a==0]=-1

    # NaN check to remove choice trials, necessary for some PPC sessions
    temp_da = ds_2s.spks.isel(trial=choice_trials)#,cell=slice(0,3))
    neuron_df = ds_2s.spks.to_pandas().reset_index(drop = True)
    neuron_df_choice = temp_da.to_pandas().reset_index(drop = True)
    neuron_isnan = np.isnan(neuron_df_choice.loc[:,1])
    if sum(neuron_isnan)>0:
        print('NaN data choice trials detected. Using only choice trials with data')
        choice_trials[np.isnan(neuron_df.loc[:,1])]=False
        temp_da = ds_2s.spks.isel(trial=choice_trials)#,cell=slice(0,3))
        neuron_df_choice = temp_da.to_pandas().reset_index(drop = True)

    # Full session
    a_hist_c = a_hist[choice_trials,:]
    R_hist_c = R_hist[choice_trials,:]
    uR_hist_c = uR_hist[choice_trials,:]
    c0 = left_trials[choice_trials]
    
    print('null models, ', end =' ')
    reg_mdl = neuron_df_choice.apply(apply_fit_CV_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, 'null']))
    regression_temp = reg_mdl.iloc[-22:].T
    reg_mdl = neuron_df_choice.apply(apply_fit_CV_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, 'Rp1']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-22:].T, ignore_index=False)
    
    print('exponential models,' , end =' ')
    reg_mdl = neuron_df_choice.apply(apply_fit_CV_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, 'exp_r']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-22:].T, ignore_index=False)
    reg_mdl = neuron_df_choice.apply(apply_fit_CV_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, 'exp_ru']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-22:].T, ignore_index=False)
    
    print('hyperbolic models,' , end =' ')
    reg_mdl = neuron_df_choice.apply(apply_fit_CV_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, 'hyp_r']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-22:].T, ignore_index=False)
    reg_mdl = neuron_df_choice.apply(apply_fit_CV_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, 'hyp_ru']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-22:].T, ignore_index=False)

    print('mixed models,' , end =' ')
    reg_mdl = neuron_df_choice.apply(apply_fit_CV_cell, axis=0,
                                     args=([c0, a_hist_c, R_hist_c, uR_hist_c, 'exp_r_hyp_u']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-22:].T, ignore_index=False)
    reg_mdl = neuron_df_choice.apply(apply_fit_CV_cell, axis=0,
                                     args=([c0, a_hist_c, R_hist_c, uR_hist_c, 'hyp_r_exp_u']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-22:].T, ignore_index=False)
    print('done')
    
    Save outputs
    output_multi = regression_temp.reset_index().set_index(['mdl_type','cell'])  
    output_xarray = output_multi.to_xarray()
    save_filename = pjoin(cellfits_dir,'ready_cellfits','cv_compare_mdl','sse_{}_15hist.nc'.format(sessions_all[ss]))
    output_xarray.to_netcdf(save_filename) 

  0%|          | 0/2 [00:00<?, ?it/s]

Checking analysis for RH055 on 180715
null models,  Checking analysis for RH055 on 180807
null models,  

## Halves tau estimation

### Functions

#### calculate p-values

In [53]:
def estimate_pval(temp_params, y, c0, a_hist_c, R_hist_c, uR_hist_c, mdl_name):
    #mdl_name = mdl_dict['mdl_name']
    y = y.values
    if (mdl_name=='null'):
        predictors = pd.DataFrame({'c0': c0})
    elif (mdl_name=='Rp1'):
        past_rc = np.multiply(a_hist_c, R_hist_c)
        rc_weights = past_rc[:,-1]
        predictors = pd.DataFrame({'RCp1': rc_weights,
                                       'c0': c0})
    elif (mdl_name=='exp_r'):
        rc_tau = np.exp(temp_params[1])
        rc_weights = BD.exp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        predictors = pd.DataFrame({'exp_rc': rc_weights,
                                       'c0': c0})
    elif (mdl_name=='hyp_r'):
        rc_tau = np.exp(temp_params[1])
        rc_weights = BD.hyp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        predictors = pd.DataFrame({'hyp_rc': rc_weights,
                                       'c0': c0})
    elif (mdl_name=='exp_ru') | (mdl_name=='exp_ru_con'):
        rc_tau = np.exp(temp_params[1])
        uc_tau = np.exp(temp_params[3])
        rc_weights = BD.exp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        uc_weights = BD.exp_filter(a_hist_c, R_hist_c, uc_tau,'ur')
        predictors = pd.DataFrame({'exp_rc': rc_weights, 'exp_uc': uc_weights,
                                   'c0': c0})
    elif (mdl_name=='hyp_ru') | (mdl_name=='hyp_ru_con'):
        rc_tau = np.exp(temp_params[1])
        uc_tau = np.exp(temp_params[3])
        rc_weights = BD.hyp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        uc_weights = BD.hyp_filter(a_hist_c, R_hist_c, uc_tau,'ur')
        predictors = pd.DataFrame({'hyp_rc': rc_weights, 'hyp_uc': uc_weights,
                                   'c0': c0})
    elif (mdl_name=='exp_r_hyp_u'):
        rc_tau = np.exp(temp_params[1])
        uc_tau = np.exp(temp_params[3])
        rc_weights = BD.exp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        uc_weights = BD.hyp_filter(a_hist_c, R_hist_c, uc_tau,'c')
        predictors = pd.DataFrame({'exp_rc': rc_weights, 'hyp_uc': uc_weights,
                                   'c0': c0})
    elif (mdl_name=='hyp_r_exp_u'):
        rc_tau = np.exp(temp_params[1])
        uc_tau = np.exp(temp_params[3])
        rc_weights = BD.hyp_filter(a_hist_c, R_hist_c, rc_tau,'r')
        uc_weights = BD.exp_filter(a_hist_c, R_hist_c, uc_tau,'c')
        predictors = pd.DataFrame({'hyp_rc': rc_weights, 'exp_uc': uc_weights,
                                   'c0': c0})
    # print(len(y), len(predictors))    
    X2 = sm.add_constant(predictors) # Add constant term to predictor dataframe
    est = sm.OLS(y, X2).fit() # Fit linear regression with ordinary least squares
    p_values = est.pvalues
    return p_values

#### pandas apply (decay)

In [54]:
def apply_fit_cell(column,c0,a_hist_c,R_hist_c,uR_hist_c,half_id,mdl_name):
    # mdl_name: 'exp_r' 'hyp_ruc' 'exp_r_con'
    rng = np.random.default_rng(220804)
    y = column
    n_obs = len(y)
    fit_func = eval('BD.reg_' + mdl_name + '_c0')
    
    if (mdl_name=='null'):
        n_params = 2; n_pvals = 2
        initial_params = [0.01, 0.01]
        bounds = [[-np.inf,np.inf],[-np.inf,np.inf]]
    elif (mdl_name=='Rp1'):
        n_params = 3; n_pvals = 3
        initial_params = [0.01, 0.01, 0.01]
        bounds = [[-np.inf,np.inf],[-np.inf,np.inf],[-np.inf,np.inf]]
    elif (mdl_name=='exp_r') | (mdl_name=='hyp_r'):
        n_params = 4; n_pvals = 3
        initial_params = [0.01, 1, 0.01, 0.01]
        bounds = [[-np.inf,np.inf],[-np.log(100),np.log(100)],[-np.inf,np.inf],[-np.inf,np.inf]]
    elif (mdl_name=='exp_ru') | (mdl_name=='hyp_ru'):
        n_params = 6; n_pvals = 4
        initial_params = [0.01, 1, -0.01, 1, 0.01, 0.01]
        bounds = [[-np.inf,np.inf],[-np.log(100),np.log(100)],[-np.inf,np.inf],[-np.log(100),np.log(100)],
                  [-np.inf,np.inf],[-np.inf,np.inf]]
    elif (mdl_name=='exp_r_hyp_u') | (mdl_name=='hyp_r_exp_u'):
        n_params = 6; n_pvals = 4
        initial_params = [0.01, 1, -0.01, 1, 0.01, 0.01]
        bounds = [[-np.inf,np.inf],[-np.log(100),np.log(100)],[-np.inf,0],[-np.log(100),np.log(100)],
                  [-np.inf,np.inf],[-np.inf,np.inf]]
    else:
        print('unable to parse mdl_type')
    
    # Simple fit
    lik_model = minimize(fit_func, initial_params, bounds=bounds, method='L-BFGS-B',
                 args=(y, c0, a_hist_c, R_hist_c, uR_hist_c, 1))
    temp_params = lik_model.x
    p_values = estimate_pval(temp_params, y, c0, a_hist_c, R_hist_c, uR_hist_c, mdl_name)
    _,sse,rsq,snr = fit_func(temp_params, y, c0, a_hist_c, R_hist_c, uR_hist_c, 0)
    loglik, norm_loglik = BD.calc_mdl_loglik(sse, n_obs)
    aic = BD.calc_aic_loglik(loglik, n_params)
    
    column['half'] = half_id
    column['mdl_type'] = mdl_name
    column['rsq'] = rsq
    column['snr'] = snr
    column['aic'] = aic
    column['loglik'] = loglik
    column['norm_loglik'] = norm_loglik
    if (mdl_name=='null'):
        column['beta_RC'] =  np.nan
        column['tau_RC'] =  np.nan
        column['beta_UC'] = np.nan
        column['tau_UC'] = np.nan
        column['beta_C'] = np.nan
        column['tau_C'] = np.nan
        column['beta_C0'] = temp_params[0]
        column['beta_0'] = temp_params[1]
        column['p_beta_RC'] = np.nan
        column['p_beta_UC'] = np.nan
        column['p_beta_C'] = np.nan
        column['p_beta_C0'] = p_values[1]
        column['p_beta_0'] = p_values[0]
    elif (mdl_name=='Rp1'):
        column['beta_RC'] =  temp_params[0]
        column['tau_RC'] =  np.nan
        column['beta_UC'] = np.nan
        column['tau_UC'] = np.nan
        column['beta_C'] = np.nan
        column['tau_C'] = np.nan
        column['beta_C0'] = temp_params[1]
        column['beta_0'] = temp_params[2]
        column['p_beta_RC'] = p_values[1]
        column['p_beta_UC'] = np.nan
        column['p_beta_C'] = np.nan
        column['p_beta_C0'] = p_values[2]
        column['p_beta_0'] = p_values[0]
    elif (mdl_name=='exp_r') | (mdl_name=='hyp_r'):
        column['beta_RC'] =  temp_params[0]
        column['tau_RC'] =  np.exp(temp_params[1])
        column['beta_UC'] = np.nan
        column['tau_UC'] = np.nan
        column['beta_C'] = np.nan
        column['tau_C'] = np.nan
        column['beta_C0'] = temp_params[2]
        column['beta_0'] = temp_params[3]
        column['p_beta_RC'] = p_values[1]
        column['p_beta_UC'] = np.nan
        column['p_beta_C'] = np.nan
        column['p_beta_C0'] = p_values[2]
        column['p_beta_0'] = p_values[0]
    elif (mdl_name=='exp_ru') | (mdl_name=='hyp_ru') | (mdl_name=='exp_r_hyp_u') | (mdl_name=='hyp_r_exp_u'):
        column['beta_RC'] =  temp_params[0]
        column['tau_RC'] =  np.exp(temp_params[1])
        column['beta_UC'] = temp_params[2]
        column['tau_UC'] = np.exp(temp_params[3])
        column['beta_C'] = np.nan
        column['tau_C'] = np.nan 
        column['beta_C0'] = temp_params[4]
        column['beta_0'] = temp_params[5]
        column['p_beta_RC'] = p_values[1]
        column['p_beta_UC'] = p_values[2]
        column['p_beta_C'] = np.nan
        column['p_beta_C0'] = p_values[3]
        column['p_beta_0'] = p_values[0]
    
    return column

#### iterate across pandas apply

In [55]:
def fit_models_half(neuron_df, c0, a_hist_c, R_hist_c, half_id):
    print('null models, ', end =' ')
    reg_mdl = neuron_df.apply(apply_fit_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, half_id, 'null']))
    regression_temp = reg_mdl.iloc[-20:].T
    reg_mdl = neuron_df.apply(apply_fit_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, half_id, 'Rp1']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-20:].T, ignore_index=False)
    
    print('exponential models,' , end =' ')
    reg_mdl = neuron_df.apply(apply_fit_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, half_id, 'exp_r']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-20:].T, ignore_index=False)
    reg_mdl = neuron_df.apply(apply_fit_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, half_id, 'exp_ru']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-20:].T, ignore_index=False)
    reg_mdl = neuron_df.apply(apply_fit_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, half_id, 'exp_r_hyp_u']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-20:].T, ignore_index=False)

    print('hyperbolic models,' , end =' ')
    reg_mdl = neuron_df.apply(apply_fit_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, half_id, 'hyp_r']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-20:].T, ignore_index=False)
    reg_mdl = neuron_df.apply(apply_fit_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, half_id, 'hyp_ru']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-20:].T, ignore_index=False)
    reg_mdl = neuron_df.apply(apply_fit_cell, axis=0,
                              args=([c0, a_hist_c, R_hist_c, uR_hist_c, half_id, 'hyp_r_exp_u']))
    regression_temp = regression_temp.append(reg_mdl.iloc[-20:].T, ignore_index=False)

    print('done')
    
    return regression_temp

### Process Data

#### Estimate across sessions

In [56]:
sessions_all = BD.get_sessions('all')
for ss in trange(len(sessions_all)):
    mouse = sessions_all[ss][:5]
    date = sessions_all[ss][6:12]
    session = sessions_all[ss][6:]
    print('Checking analysis for {} on {}'.format(mouse, date))

    ds_2s = xr.open_dataset(pjoin(imaging_dir,'preprocess_data_2s','{}_ready_2s.nc'.format(sessions_all[ss])))

    # Prepare behavior information
    a = np.array(ds_2s.a)
    R = np.array(ds_2s.R)
    n_back = 15
    a_hist,R_hist,uR_hist = BD.prepare_hist_matrix(a,R,n_back)
    a_cleaned = a.copy()+0.
    a_cleaned[a>2] = 0
    a_cleaned[a==2] = -1

    # Choice metrics
    choice_trials = (a==1)|(a==2)
    left_trials = (a==1)+0
    left_trials[a==0]=-1

    # NaN check to remove choice trials, necessary for some PPC sessions
    temp_da = ds_2s.spks.isel(trial=choice_trials)#,cell=slice(0,3))
    neuron_df = ds_2s.spks.to_pandas().reset_index(drop = True)
    neuron_df_choice = temp_da.to_pandas().reset_index(drop = True)
    neuron_isnan = np.isnan(neuron_df_choice.loc[:,1])
    if sum(neuron_isnan)>0:
        print('NaN data choice trials detected. Using only choice trials with data')
        choice_trials[np.isnan(neuron_df.loc[:,1])]=False
        temp_da = ds_2s.spks.isel(trial=choice_trials)#,cell=slice(0,3))
        neuron_df_choice = temp_da.to_pandas().reset_index(drop = True)

    # Full session
    a_hist_c = a_hist[choice_trials,:]
    R_hist_c = R_hist[choice_trials,:]
    uR_hist_c = uR_hist[choice_trials,:]
    c0 = left_trials[choice_trials]
    regression_output = fit_models_half(neuron_df_choice, c0, a_hist_c, R_hist_c, 0)
      
    # Select Half of choice trials
    choice_inds = np.where(choice_trials)[0]
    num_choice = sum(choice_trials)
    half_choices = np.floor(num_choice/2).astype(int) 
    sub_trials_1 = choice_inds[:half_choices]
    sub_trials_2 = choice_inds[half_choices:]
    # Half 1
    a_hist_c = a_hist[sub_trials_1,:]
    R_hist_c = R_hist[sub_trials_1,:]
    uR_hist_c = uR_hist[sub_trials_1,:]
    c0 = a_cleaned[sub_trials_1]
    temp_da = ds_2s.spks.isel(trial=sub_trials_1)#,cell=slice(0,3))
    neuron_df_choice = temp_da.to_pandas().reset_index(drop = True)
    regression_temp = fit_models_half(neuron_df_choice, c0, a_hist_c, R_hist_c, 1)
    regression_output = regression_output.append(regression_temp)
    
    # Half 2
    a_hist_c = a_hist[sub_trials_2,:]
    R_hist_c = R_hist[sub_trials_2,:]
    uR_hist_c = uR_hist[sub_trials_2,:]
    c0 = a_cleaned[sub_trials_2]
    temp_da = ds_2s.spks.isel(trial=sub_trials_2)#,cell=slice(0,3))
    neuron_df_choice = temp_da.to_pandas().reset_index(drop = True)
    regression_temp = fit_models_half(neuron_df_choice, c0, a_hist_c, R_hist_c, 2)
    regression_output = regression_output.append(regression_temp)
    
    # Save outputs
    output_multi = regression_output.reset_index().set_index(['half','mdl_type','cell'])  
    output_xarray = output_multi.to_xarray()
    save_filename = pjoin(cellfits_dir,'ready_cellfits','halves_compare_mdl','sse_{}_15hist.nc'.format(sessions_all[ss]))
    # output_xarray.to_netcdf(save_filename) 

  0%|          | 0/2 [00:00<?, ?it/s]

Checking analysis for RH055 on 180715
null models,  exponential models, hyperbolic models, done
null models,  exponential models, hyperbolic models, done
null models,  exponential models, hyperbolic models, done
Checking analysis for RH055 on 180807
null models,  exponential models, hyperbolic models, done
null models,  exponential models, hyperbolic models, done
null models,  exponential models, hyperbolic models, done


## Quasihyperbolic analysis

### Functions

#### load data function

In [65]:
def load_cell_data(session):
    save_filename = pjoin(cellfits_dir,'ready_cellfits','halves_compare_mdl',
                          'sse_{}_15hist.nc'.format(session))
    
    r_xarray = xr.open_dataset(save_filename)
    crit_bool = ( (r_xarray.sel(mdl_type='exp_ru',half=1).p_beta_RC<0.05) &
                  (r_xarray.sel(mdl_type='exp_ru',half=2).p_beta_RC<0.05) &
                  (r_xarray.sel(mdl_type='exp_ru',half=0).tau_RC<99)&
                  (r_xarray.sel(mdl_type='exp_ru',half=0).tau_RC>0.01) )
    
    out_pd = r_xarray.sel(mdl_type='exp_ru',half=0).to_dataframe()[['beta_RC','tau_RC']]
    out_pd['crit'] = crit_bool
    return out_pd

#### modelfit weighted neusum

In [87]:
# for a given session
def modelfit_quasilogit(a,R,n_tau,drawn_tau):
    n_back = 15
    # Prepare history matrices
    a_hist,R_hist,_ = BD.prepare_hist_matrix(a,R,n_back) 
    
    n_draw_iter = np.floor(n_draw/n_tau).astype(int); #print(n_tau,n_draw_iter)
    drawn_tau_mat = np.reshape(drawn_tau[:n_draw_iter*n_tau], (n_draw_iter,n_tau))
    initialize_bool = True
    for dd in range(n_draw_iter):
        tau_vec = tau_vec = drawn_tau_mat[dd,:]
    
        # Prepare the model initial guesses and parameters
        filter_initguess = np.random.rand(n_tau)
        filter_initguess[filter_initguess<1e-3] = 1e-3 # to ensure no 0 value
        init_params = np.append(0,filter_initguess) # add constant
        bounds = np.vstack([np.append(-np.inf,0*np.ones(len(tau_vec))),
                            np.inf*np.ones(len(init_params))]).T     

        n_params = len(filter_initguess)
        # Run k-fold cross validation
        cv_iter = 0
        loglik_vec = np.zeros(10)
        aic_vec = np.zeros(10)
        pa_vec = np.zeros(10)
        kf = KFold(n_splits=10, shuffle=False)#, random_state=0)
        kf.get_n_splits(np.arange(0,len(a)))
        for train_index, test_index in kf.split(np.arange(0,len(a))):  
            minimizer_kwargs = {'method':'L-BFGS-B', 'bounds':bounds, 'args':(
                tau_vec, a[train_index], a_hist[train_index,:], R_hist[train_index,:], 1)}
            with np.errstate(divide='ignore',invalid='ignore'):
                try:
                    lik_model = basinhopping(BD.logit_quasi_fit, init_params, minimizer_kwargs=minimizer_kwargs, niter=3)
                except:# Necessary to stop weird x0 error
                    init_params = 0.5*np.ones(len(init_params))
                    lik_model = basinhopping(BD.logit_quasi_fit, init_params, minimizer_kwargs=minimizer_kwargs, niter=3)
            mdl_weights = lik_model.x
            p_logit, _, _, nloglik = BD.logit_quasi_fit(mdl_weights, tau_vec,
                                                   a[test_index], 
                                                   a_hist[test_index,:], 
                                                   R_hist[test_index,:], 0)
            pa_vec[cv_iter],_ = BD.PA_LL_logit(a[test_index],p_logit)
            aic_vec[cv_iter] = 2*n_params + 2*nloglik
            loglik_vec[cv_iter] = -nloglik
            cv_iter+=1
        
        temp_series = pd.Series({'n_tau': n_tau, 'draw_iter': dd, 
                                 'loglik': loglik_vec.mean(), 'loglik_trial': loglik_vec.mean()/len(test_index), 
                                 'pa': pa_vec.mean(),
                                 'aic': aic_vec.mean()})
        if initialize_bool:
            temp_df = pd.DataFrame(temp_series).T
            initialize_bool = False
        else:
            temp_df = temp_df.append(temp_series, ignore_index=True)
    
    return temp_df

### Load behavior fits

In [84]:
file_savename = pjoin(behavior_dir, 'behlogit_exp_hyp_history.nc')
behlogit_expert = xr.open_dataset(file_savename)
behlogit_fit = behlogit_expert.sel(mdl_name=['exp_r','hyp_r'],mdl_history=15).drop('mdl_history')[[
    'CV_mean_loglik', 'CV_mean_aic', 'CV_mean_pa']]

### Fit weighted quasi-hyperbolic to behavior, given tau

In [88]:
tqdm.pandas()
n_draw = 150
max_n_tau = 15 # max number of tau to draw from distribution, iterated up to this number

# Load data
beh_df = pd.read_pickle(pjoin(behavior_dir,'all_behavior_df.pkl'))

initialize_df = True # True False
for area in ['RSC']:#,'PPC','ALM','M2','S1']: # ,,'V1']
    print('Running for area {}'.format(area))
    generating_area = area
    sessions_all = BD.get_sessions(area)
    output_all = pd.DataFrame([])
    for ss in range(len(sessions_all)):
        out_pd = load_cell_data(sessions_all[ss])
        output_all = output_all.append(out_pd,ignore_index=True)
    tau_distribution = output_all.loc[output_all['crit'],'tau_RC'].values
    np.random.seed(211021) # n_draw_iteration
    drawn_tau = np.random.choice(tau_distribution,n_draw)
    for nn in trange(max_n_tau): 
        n_tau = nn+1 # number of tau used to produce QH
        # print('Running with {} taus'.format(n_tau))
        for session_id in range(len(beh_df)):
            # Prepare behavior
            n_back = 15 
            a = beh_df.loc[session_id,'a']
            R = beh_df.loc[session_id,'R']
            # Run CV fit
            cv_fit_df = modelfit_quasilogit(a,R,n_tau,drawn_tau)
            # Add fit metadata
            cv_fit_df['generating_area']= generating_area
            cv_fit_df['session_id']= session_id
            # Add fit comparisons
            ll_exp = behlogit_fit.sel(mdl_name='exp_r').isel(session_id=session_id).CV_mean_loglik.item()
            ll_hyp = behlogit_fit.sel(mdl_name='hyp_r').isel(session_id=session_id).CV_mean_loglik.item()
            aic_exp = behlogit_fit.sel(mdl_name='exp_r').isel(session_id=session_id).CV_mean_aic.item()
            aic_hyp = behlogit_fit.sel(mdl_name='hyp_r').isel(session_id=session_id).CV_mean_aic.item()
            pa_exp = behlogit_fit.sel(mdl_name='exp_r').isel(session_id=session_id).CV_mean_pa.item()
            pa_hyp = behlogit_fit.sel(mdl_name='hyp_r').isel(session_id=session_id).CV_mean_pa.item()
            cv_fit_df['loglik_dexp'] = cv_fit_df['loglik'] - ll_exp
            cv_fit_df['loglik_dhyp'] = cv_fit_df['loglik'] - ll_hyp
            cv_fit_df['aic_dexp'] = cv_fit_df['aic'] - aic_exp
            cv_fit_df['aic_dhyp'] = cv_fit_df['aic'] - aic_hyp
            cv_fit_df['pa_dexp'] = cv_fit_df['pa'] - pa_exp
            cv_fit_df['pa_dhyp'] = cv_fit_df['pa'] - pa_hyp
            
            if initialize_df:
                performance_cv_df = cv_fit_df
                initialize_df = False
            else:
                performance_cv_df = performance_cv_df.append(cv_fit_df, ignore_index=True)
                
    output_xarray = performance_cv_df.set_index(['generating_area','n_tau','session_id','draw_iter']).to_xarray()
    
    file_savename = pjoin(behavior_dir,'quasihyp_subsample_cv.nc')
    output_xarray.to_netcdf(file_savename) 

Running for area RSC


  0%|          | 0/5 [00:00<?, ?it/s]

# Opto analysis

## General import functions

In [4]:
%reset -f
import sys
from os.path import dirname, join as pjoin
from os import listdir
sys.path.append('C:\\jupyter_notebooks\\Danskin_SciAdv_2023\\py_code') # set local directory
import numpy as np
import pickle
import random
import xarray as xr
import pandas as pd

import h5py
import scipy.io as sio
from scipy import stats
import statsmodels as sm # import statsmodels.api as sm
from scipy.optimize import minimize, basinhopping, curve_fit
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm, trange

In [5]:
## Directories
%autoreload
import bdanskin as BD 

project_dir = 'C:\\jupyter_notebooks\\Danskin_SciAdv_2023' # local directory
opto_dir = pjoin(project_dir, 'chr2_optogenetics')
behavior_dir = pjoin(project_dir, 'hattori_datasets_behavior')

## Winstay, loseswitch

### Functions

#### pandas winstay, loseswitch

In [6]:
def ws_ls_opto_metrics_pd(row):
    inac,a,R = row[['Inac Type','a','R']]
    # print(a, R)
    if inac=='ra':
        opto = row['Opto']
    else:
        opto = row['Opto Shift']
        
    pre_opto = np.append(opto[1:],0)
    post_opto = np.append(0,opto[:-1])  
  
    choice = (a==1)+0
    choice[a==2] = -1
    previous_choice = np.append(0, choice[:-1])
    stay_vec = ((choice==previous_choice) & choice!=0)+0
    switch_vec = ((choice!=previous_choice) & (choice!=0) & (previous_choice!=0))+0
    post_win = np.append(0, R[:-1])
    post_lose = np.append(0, R[:-1]==0)
    post_win[choice==0] = 0
    post_lose[choice==0] = 0
    post_lose[previous_choice==0] = 0
    
    p_switch_ctrl = sum(switch_vec  & (opto==0))/sum((choice!=0) & (previous_choice!=0) & (opto==0))
    p_switch_opto = sum(switch_vec  & (opto==1))/sum((choice!=0) & (previous_choice!=0) & (opto==1))
    p_switch_preopto = sum(switch_vec  & (pre_opto==1))/sum((choice!=0) & (previous_choice!=0) & (pre_opto==1))
    p_switch_postopto = sum(switch_vec  & (post_opto==1))/sum((choice!=0) & (previous_choice!=0) & (post_opto==1))

    p_stay_ctrl = sum(stay_vec & (opto==0))/sum((choice!=0) & (previous_choice!=0) & (opto==0))
    p_stay_opto = sum(stay_vec & (opto==1))/sum((choice!=0) & (previous_choice!=0) & (opto==1))
    p_stay_preopto = sum(stay_vec & (pre_opto==1))/sum((choice!=0) & (previous_choice!=0) & (pre_opto==1))
    p_stay_postopto = sum(stay_vec & (post_opto==1))/sum((choice!=0) & (previous_choice!=0) & (post_opto==1))

    ws_ctrl = sum(post_win & stay_vec & (opto==0)) / sum(post_win & (opto==0))
    ws_opto = sum(post_win & stay_vec & (opto==1)) / sum(post_win & (opto==1))
    ws_preopto = sum(post_win & stay_vec & (pre_opto==0)) / sum(post_win & (pre_opto==0))
    ws_postopto = sum(post_win & stay_vec & (post_opto==0)) / sum(post_win & (post_opto==0))

    ws_ctrl = sum(post_win & stay_vec & (opto==0)) / sum(post_win & (opto==0))
    ws_opto = sum(post_win & stay_vec & (opto==1)) / sum(post_win & (opto==1))
    ws_preopto = sum(post_win & stay_vec & (pre_opto==0)) / sum(post_win & (pre_opto==0))
    ws_postopto = sum(post_win & stay_vec & (post_opto==0)) / sum(post_win & (post_opto==0))

    ls_ctrl = sum(post_lose & switch_vec & (opto==0)) / sum(post_lose & (opto==0))
    ls_opto = sum(post_lose & switch_vec & (opto==1)) / sum(post_lose & (opto==1))
    ls_preopto = sum(post_lose & switch_vec & (pre_opto==0)) / sum(post_lose & (pre_opto==0))
    ls_postopto = sum(post_lose & switch_vec & (post_opto==0)) / sum(post_lose & (post_opto==0))
    
    row['p_stay_ctrl'] = p_stay_ctrl
    row['p_stay_preopto'] = p_stay_preopto
    row['p_stay_opto'] = p_stay_opto
    row['p_stay_postopto'] = p_stay_postopto
    row['p_switch_ctrl'] = p_switch_ctrl
    row['p_switch_preopto'] = p_switch_preopto
    row['p_switch_opto'] = p_switch_opto
    row['p_switch_postopto'] = p_switch_postopto
    row['ws_ctrl'] = ws_ctrl
    row['ws_preopto'] = ws_preopto
    row['ws_opto'] = ws_opto
    row['ws_postopto'] = ws_postopto
    row['ls_ctrl'] = ls_ctrl
    row['ls_preopto'] = ls_preopto
    row['ls_opto'] = ls_opto
    row['ls_postopto'] = ls_postopto
    row['ws_stay_ctrl'] = ws_ctrl/p_stay_ctrl
    row['ws_stay_preopto'] = ws_preopto/p_stay_preopto
    row['ws_stay_opto'] = ws_opto/p_stay_opto
    row['ws_stay_postopto'] = ws_postopto/p_stay_postopto
    row['ls_switch_ctrl'] = ls_ctrl/p_switch_ctrl
    row['ls_switch_preopto'] = ls_preopto/p_switch_preopto
    row['ls_switch_opto'] = ls_opto/p_switch_opto
    row['ls_switch_postopto'] = ls_postopto/p_switch_postopto
    
    row['ws_stay_ctrl_prime'] = ws_ctrl/np.mean([ws_ctrl, 1-ls_ctrl])
    row['ws_stay_preopto_prime'] = ws_preopto/np.mean([ws_preopto, 1-ls_preopto])
    row['ws_stay_opto_prime'] = ws_opto/np.mean([ws_opto, 1-ls_opto])
    row['ws_stay_postopto_prime'] = ws_postopto/np.mean([ws_postopto, 1-ls_postopto])
    row['ls_switch_ctrl_prime'] = ls_ctrl/np.mean([ls_ctrl, 1-ws_ctrl])
    row['ls_switch_preopto_prime'] = ls_preopto/np.mean([ls_preopto, 1-ws_preopto])
    row['ls_switch_opto_prime'] = ls_opto/np.mean([ls_opto, 1-ws_opto])
    row['ls_switch_postopto_prime'] = ls_postopto/np.mean([ls_postopto, 1-ws_postopto])

    return row

### Load data

In [7]:
# Load session summary
chr2_session_summary = pd.read_pickle(pjoin(opto_dir,'ChR2_session_summary.pkl'))

### Iterate across model fits

In [8]:
tqdm.pandas()

fit_df = chr2_session_summary.progress_apply(ws_ls_opto_metrics_pd, axis=1)
chr2_ws_ls = fit_df.drop(columns=['a','R','Opto','Opto Shift'])

chr2_ws_ls.to_pickle(pjoin(opto_dir, 'ChR2_wsls.pkl'))

  0%|          | 0/255 [00:00<?, ?it/s]

## Logistic regression

### Functions

#### pandas logistic modelfit (subsample 1000x)

In [102]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression

In [103]:
def modelfit_opto_logistic_subsample_pd(row, mdl_type, reg_type):
    rng = np.random.default_rng(220727)
    # rng = np.random.default_rng(100000) 
    
    n_back = 5
    inac,a,R = row[['Inac Type','a','R']]
    if inac=='ra':
        Opto = row['Opto']
    else:
        Opto = row['Opto Shift']   
    n_choice = sum((a==1)|(a==2))
    choice_trials = (a==1)|(a==2)
    
    predictor_df = BD.prepare_opto_predictor_df(a,R,Opto)
    glm_mat, glm_target, opto_param,_  = BD.prepare_opto_glm_5(predictor_df, mdl_type)
    
    # Subsample
    n_iterations = 1000
    sample_size = 0.9
    opto_inds = np.where(opto_param)[0]
    ctrl_inds = np.where(opto_param==0)[0]
    n_opto = len(opto_inds)
    n_per_iteration = np.floor(n_opto*sample_size).astype(int)
    n_per_test = n_opto-n_per_iteration
    
    for nn in range(n_iterations):
        perm_ctrl = rng.permutation(ctrl_inds)
        perm_opto = rng.permutation(opto_inds)
        ctrl_train = perm_ctrl[:n_per_iteration]
        ctrl_test = perm_ctrl[n_per_iteration:n_per_iteration+n_per_test]
        opto_train = perm_opto[:n_per_iteration]
        opto_test = perm_opto[n_per_iteration:n_per_iteration+n_per_test]
        
        train_inds = np.concatenate([opto_train, ctrl_train])
        train_inds.sort()
        test_inds = np.concatenate([opto_test, ctrl_test])
        test_inds.sort()
        n_test = len(test_inds)
        
        # Estimate regression
        if reg_type=='none':
            log_reg = LogisticRegression(solver='newton-cg', penalty='none', n_jobs=-1, random_state=0)
        else: 
            log_reg = LogisticRegressionCV(solver='saga', cv=5, penalty=reg_type, n_jobs=-1, random_state=0, refit=True)
        fit_mdl = log_reg.fit(glm_mat[train_inds,:],glm_target[train_inds])    
        
        coef = fit_mdl.coef_[0]
        intercept = fit_mdl.intercept_[0]
        temp_coef = coef.copy()
        temp_coef[-1] = temp_coef[-1]+intercept
        mdl_coef = np.append(temp_coef, intercept)

        loglik_train, pa_train = BD.logistic_calc_loglik_pa(glm_mat[train_inds,:], glm_target[train_inds], mdl_coef)
        loglik_test, pa_test = BD.logistic_calc_loglik_pa(glm_mat[test_inds,:], glm_target[test_inds], mdl_coef)
        oll_0, cll_0, opa_0, cpa_0 = BD.logistic_opto_calc_loglik(glm_mat[train_inds,:], glm_target[train_inds], mdl_coef)
        oll_1, cll_1, opa_1, cpa_1 = BD.logistic_opto_calc_loglik(glm_mat[test_inds,:], glm_target[test_inds], mdl_coef)
            
        if nn==0:
            all_coef_mat = mdl_coef.copy()
            ll_train_all= loglik_train.copy()
            pa_train_all = pa_train.copy()
            ll_test_all= loglik_test.copy()
            pa_test_all = pa_test.copy()
            ll_train_ctrl = cll_0.copy()
            ll_train_opto = oll_0.copy()
            pa_train_ctrl = cpa_0.copy()
            pa_train_opto = opa_0.copy()
            ll_test_ctrl = cll_1.copy()
            ll_test_opto = oll_1.copy()
            pa_test_ctrl = cpa_1.copy()
            pa_test_opto = opa_1.copy()

        else:
            all_coef_mat = np.vstack((all_coef_mat, mdl_coef))
            ll_train_all = np.append(ll_train_all, loglik_train)
            pa_train_all = np.append(pa_train_all, pa_train)
            ll_test_all = np.append(ll_test_all, loglik_test)
            pa_test_all = np.append(pa_test_all, pa_test)
            ll_train_ctrl = np.append(ll_train_ctrl, cll_0)
            ll_train_opto = np.append(ll_train_opto, oll_0)
            pa_train_ctrl = np.append(pa_train_ctrl, cpa_0)
            pa_train_opto = np.append(pa_train_opto, opa_0)
            ll_test_ctrl = np.append(ll_test_ctrl, cll_1)
            ll_test_opto = np.append(ll_test_opto, oll_1)
            pa_test_ctrl = np.append(pa_test_ctrl, cpa_1)
            pa_test_opto = np.append(pa_test_opto, opa_1)
    
    ll_train_all[ll_train_all==-np.inf] = np.nan
    ll_test_all[ll_test_all==-np.inf] = np.nan
    ll_train_ctrl[ll_train_ctrl==-np.inf] = np.nan
    ll_train_opto[ll_train_opto==-np.inf] = np.nan
    ll_test_ctrl[ll_test_ctrl==-np.inf] = np.nan
    ll_test_opto[ll_test_opto==-np.inf] = np.nan
    frac_opto_neg = calc_bootstrap_onetail(all_coef_mat, mdl_type)
    
    row['mdl_type'] = mdl_type
    row['mdl_hist'] = n_back
    row['mdl_penalty'] = reg_type
    row['mdl_subsample'] = n_iterations
    row['mean_weights'] = np.nanmean(all_coef_mat,0)
    row['median_weights'] = np.nanmedian(all_coef_mat,0)
    row['boot_opto_neg'] = frac_opto_neg
    row['loglik_train'] = np.nanmean(ll_train_all)
    row['loglik_train_ctrl'] = np.nanmean(ll_train_ctrl)
    row['loglik_train_opto'] = np.nanmean(ll_train_opto)
    row['loglik_test'] = np.nanmean(ll_test_all)
    row['loglik_test_ctrl'] = np.nanmean(ll_test_ctrl)
    row['loglik_test_opto'] = np.nanmean(ll_test_opto)
    row['pa_train'] = np.nanmean(pa_train_all)
    row['pa_train_ctrl'] = np.nanmean(pa_train_ctrl)
    row['pa_train_opto'] = np.nanmean(pa_train_opto)
    row['pa_test'] = np.nanmean(pa_test_all)
    row['pa_test_ctrl'] = np.nanmean(pa_test_ctrl)
    row['pa_test_opto'] = np.nanmean(pa_test_opto)
    
    # Full session fit
    if reg_type=='none':
        log_reg = LogisticRegression(solver='newton-cg', penalty='none', n_jobs=-1, random_state=0)
    else: 
        log_reg = LogisticRegressionCV(solver='saga', cv=5, penalty=reg_type, n_jobs=-1, random_state=0, refit=True)
    fit_mdl = log_reg.fit(glm_mat,glm_target)    

    coef = fit_mdl.coef_[0]
    intercept = fit_mdl.intercept_[0]
    temp_coef = coef.copy()
    temp_coef[-1] = temp_coef[-1]+intercept
    mdl_coef = np.append(temp_coef, intercept)
    loglik_full, pa_full = BD.logistic_calc_loglik_pa(glm_mat, glm_target, mdl_coef)

    row['full_weights'] = mdl_coef
    row['loglik_full'] = loglik_full
    row['pa_full'] = pa_full
    return row

#### calculate opto stats

In [104]:
def calc_bootstrap_onetail(coef_mat, mdl_type):
    # Assumed 5-history model
    if (mdl_type=='RU')|(mdl_type=='RC'):
        delta_weights = coef_mat[:,slice(10,20)] - coef_mat[:,slice(0,10)]
        delta_weights = np.column_stack((delta_weights, (coef_mat[:,-2] - coef_mat[:,-1])))
        mean_bool_delta = np.mean(delta_weights<0,axis=0)
    elif (mdl_type=='RUC'):
        delta_weights = coef_mat[:,slice(15,30)] - coef_mat[:,slice(0,15)]
        delta_weights = np.column_stack((delta_weights, (coef_mat[:,-2] - coef_mat[:,-1])))
        mean_bool_delta = np.mean(delta_weights<0,axis=0)
        
    return mean_bool_delta

### Load data

In [113]:
# Load session summary
chr2_session_summary = pd.read_pickle(pjoin(opto_dir,'ChR2_session_summary.pkl'))

### Iterate across model fits

In [109]:
tqdm.pandas()
temp_df = chr2_session_summary#.iloc[:3]

# modelfit_opto_logistic_subsample_pd(row, mdl_type, reg_type)
print('Fitting RU model...')
fit_df = temp_df.progress_apply(modelfit_opto_logistic_subsample_pd,axis=1, args=['RU','none'])
output_reg = fit_df.drop(columns=['a','R','Opto','Opto Shift'])
fit_df = temp_df.progress_apply(modelfit_opto_logistic_subsample_pd,axis=1, args=['RU','l1'])
output_reg = output_reg.append(fit_df.drop(columns=['a','R','Opto','Opto Shift']), ignore_index=False)
fit_df = temp_df.progress_apply(modelfit_opto_logistic_subsample_pd,axis=1, args=['RU','l2'])
output_reg = output_reg.append(fit_df.drop(columns=['a','R','Opto','Opto Shift']), ignore_index=False)

print('  done!')
output_reg.to_pickle(pjoin(opto_dir, 'logistic_ChR2_l1l2_subsample1000.pkl'))

Fitting RU model...


  0%|          | 0/3 [00:00<?, ?it/s]

## Behlogits

### Functions

#### behlogits local (extra bias terms)

In [8]:
def logit_exp_r_w(params,a,a_hist,R_hist,uR_hist,session_mat,fit_bool):
    n_back = np.shape(a_hist)[1]
    beta_rc = params[0]
    tau_rc = params[1]
    const = params[2:]
    
    past_r = np.multiply(a_hist,R_hist) # already flipped
    past_ur = np.multiply(a_hist,uR_hist)
    g_rc = np.flip(np.exp(-np.arange(0,n_back)/tau_rc)) 
    g_rc[np.isnan(g_rc)]=0.  # Catch to remove nans
    if sum(g_rc)!=0:
        g_rc = g_rc/sum(g_rc) 

    # convert the sum into a probability, calculate negative log likelihood
    sum_weights = (beta_rc*(np.dot(past_r,g_rc))  + np.dot(session_mat,const))
    p_logit = 1/(1+np.exp(-sum_weights))
    nloglik = -(np.nansum(np.log(p_logit[a==1]))+np.nansum(np.log(1-p_logit[a==0])))
    
    # If this is for fitting, return only nloglik
    if fit_bool:
        return nloglik
    else:
        return p_logit, nloglik
    
def logit_hyp_r_w(params,a,a_hist,R_hist,uR_hist,session_mat,fit_bool):
    n_back = np.shape(a_hist)[1]
    beta_rc = params[0]
    tau_rc = params[1]
    const = params[2:]
    
    past_r = np.multiply(a_hist,R_hist) # already flipped
    past_ur = np.multiply(a_hist,uR_hist)
    g_rc = np.flip(1/(1+np.arange(0,n_back)/tau_rc))
    g_rc[np.isnan(g_rc)]=0.  # Catch to remove nans
    if sum(g_rc)!=0:
        g_rc = g_rc/sum(g_rc) 
    
    # convert the sum into a probability, calculate negative log likelihood
    sum_weights = (beta_rc*(np.dot(past_r,g_rc)) + np.dot(session_mat,const))
    p_logit = 1/(1+np.exp(-sum_weights))
    nloglik = -(np.nansum(np.log(p_logit[a==1]))+np.nansum(np.log(1-p_logit[a==0])))
    
    # If this is for fitting, return only nloglik
    if fit_bool:
        return nloglik
    else:
        return p_logit, nloglik

#### prepare design matrices

In [9]:
def prepare_opto_predictor_df(a,R,Opto,area):
    a_hist, R_hist, uR_hist = BD.prepare_hist_matrix(a,R,10)

    # Codify the dependent variable: choice
    dv = (a==1).astype(int)
    dv = dv.reshape([len(dv),1])

    opto_on = np.where(Opto)[0]
    choice_trials = ((a==1) | (a==2)) 
    opto_choice_trials = ((a==1) | (a==2)) & Opto
    opto_inds = np.where(opto_choice_trials)[0]
    
    opto_choice_trials = opto_choice_trials.reshape([len(opto_choice_trials),1])
    
    trials_to_use = np.zeros(len(a))
    trials_to_use[opto_inds]=1
    trials_to_use = trials_to_use.astype(bool)
    
    if area=='Control':
        trial_type = np.zeros([len(a),1])
    else:
        trial_type = np.ones([len(a),1])     

    temp_df = pd.DataFrame(np.hstack([dv[trials_to_use], 
                                      trial_type[trials_to_use],
                                      a_hist[trials_to_use,:],
                                      R_hist[trials_to_use,:],
                                      uR_hist[trials_to_use,:]]),
                  columns=['DV','opto',
                           'ap10','ap9','ap8','ap7','ap6','ap5','ap4','ap3','ap2','ap1',
                           'Rp10','Rp9','Rp8','Rp7','Rp6','Rp5','Rp4','Rp3','Rp2','Rp1',
                           'uRp10','uRp9','uRp8','uRp7','uRp6','uRp5','uRp4','uRp3','uRp2','uRp1'])

    return temp_df  

def prepare_concatenated_df_by_mouse(mouse_sub_df):
    n_sessions = len(mouse_sub_df)
    for nn in range(n_sessions):
        temp_row = mouse_sub_df.iloc[nn]
        area,inac,a,R = temp_row[['Area','Inac Type','a','R']]
        if inac=='ra':
            Opto = temp_row['Opto']
        else:
            Opto = temp_row['Opto Shift'] 
    
        temp_df = prepare_opto_predictor_df(a,R,Opto,area)
        temp_df['ss_'+str(nn)] = np.ones(len(temp_df))
        
        if nn==0:
            out_df = temp_df.copy()
        else:
            out_df = out_df.append(temp_df)
            
    out_df = out_df.fillna(0)
    out_df['s_all'] = 1
    return out_df


In [10]:
def generate_RUC(predictor_df,n_back):
    # Prepare matrices
    if n_back==10:
        temp_a_hist = predictor_df[['ap10','ap9','ap8','ap7','ap6','ap5','ap4','ap3','ap2','ap1']].to_numpy()
        temp_R_hist = predictor_df[['Rp10','Rp9','Rp8','Rp7','Rp6','Rp5','Rp4','Rp3','Rp2','Rp1']].to_numpy()
        temp_uR_hist = predictor_df[['uRp10','uRp9','uRp8','uRp7','uRp6','uRp5','uRp4','uRp3','uRp2','uRp1']].to_numpy()
    elif n_back==5:
        temp_a_hist = predictor_df[['ap5','ap4','ap3','ap2','ap1']].to_numpy()
        temp_R_hist = predictor_df[['Rp5','Rp4','Rp3','Rp2','Rp1']].to_numpy()
        temp_uR_hist = predictor_df[['uRp5','uRp4','uRp3','uRp2','uRp1']].to_numpy()
    
    session_mat = predictor_df.iloc[:,32:].to_numpy()
    glm_target = predictor_df['DV'].values
    return glm_target, temp_a_hist, temp_R_hist, temp_uR_hist, session_mat

#### modelfit function

In [47]:
def fit_subsample_behlogit(temp_beh, area, fit_method, n_iterations, n_back):
    rng = np.random.default_rng(220202)
    # opt_catch = True
    predictor_df = prepare_concatenated_df_by_mouse(temp_beh)    
    glm_target, a_hist, R_hist, uR_hist, session_mat = generate_RUC(predictor_df,n_back)
    
    # Model specs
    n_bias = np.shape(session_mat)[1]
    n_params = 2+n_bias
    initial_params = np.append([1,2],np.zeros(n_bias))
    bounds = np.array([[0,np.inf],[0,20]])
    bias_bounds = np.zeros([n_bias,2]); bias_bounds[:,0] = -np.inf; bias_bounds[:,1] = np.inf; 
    bounds = np.vstack([bounds,bias_bounds])
    
    # Subsample
    sample_size = 0.9
    opto_bool = predictor_df['opto'].values
    opto_inds = np.where(opto_bool)[0]
    ctrl_inds = np.where(opto_bool==0)[0]
    n_condition = np.min([len(opto_inds),len(ctrl_inds)])
    n_per_iteration = np.floor(n_condition*sample_size).astype(int)
    n_per_test = n_condition-n_per_iteration
    print(len(opto_inds), len(ctrl_inds),n_per_iteration)
    
    for nn in range(n_iterations):
        perm_ctrl = rng.permutation(ctrl_inds)
        perm_opto = rng.permutation(opto_inds)
        ctrl_train = perm_ctrl[:n_per_iteration]
        ctrl_test = perm_ctrl[n_per_iteration:n_per_iteration+n_per_test]
        opto_train = perm_opto[:n_per_iteration]
        opto_test = perm_opto[n_per_iteration:]
        
        # Fit on opto trials
        if fit_method=='basinhopping':
            minimizer_kwargs = {"method":'L-BFGS-B', "bounds":bounds,
                                "args": (glm_target[opto_train], a_hist[opto_train,:],
                                         R_hist[opto_train,:], uR_hist[opto_train,:], 
                                         session_mat[opto_train,:], 1)}
            
            # Fit exponential
            beh_func = logit_exp_r_w
            with np.errstate(divide='ignore',invalid='ignore'):
                lik_model = basinhopping(beh_func, initial_params, minimizer_kwargs=minimizer_kwargs, niter=10)
            opto_exp_train_params = lik_model.x
            _,opto_exp_nloglik = beh_func(opto_exp_train_params, glm_target[opto_test],
                                          a_hist[opto_test,:], R_hist[opto_test,:],
                                          uR_hist[opto_test,:], session_mat[opto_test,:],0)
            
            # Fit hyperbolic
            beh_func = logit_hyp_r_w
            with np.errstate(divide='ignore',invalid='ignore'):
                lik_model = basinhopping(beh_func, initial_params, minimizer_kwargs=minimizer_kwargs, niter=10)
            opto_hyp_train_params = lik_model.x
            _,opto_hyp_nloglik = beh_func(opto_hyp_train_params, glm_target[opto_test],
                                          a_hist[opto_test,:], R_hist[opto_test,:],
                                          uR_hist[opto_test,:], session_mat[opto_test,:],0)
        else:
            # Fit exponential
            beh_func = logit_exp_r_w
            lik_model = minimize(beh_func, initial_params, bounds=bounds, method=fit_method,
                                 args=(glm_target[opto_train], a_hist[opto_train,:],
                                       R_hist[opto_train,:], uR_hist[opto_train,:],
                                       session_mat[opto_train,:], 1))
            opto_exp_train_params = lik_model.x
            _,opto_exp_nloglik = beh_func(opto_exp_train_params, glm_target[opto_test],
                                          a_hist[opto_test,:], R_hist[opto_test,:],
                                          uR_hist[opto_test,:], session_mat[opto_test,:],0)
            
            # Fit hyperbolic
            beh_func = logit_hyp_r_w
            lik_model = minimize(beh_func, initial_params, bounds=bounds, method=fit_method,
                                 args=(glm_target[opto_train], a_hist[opto_train,:],
                                       R_hist[opto_train,:], uR_hist[opto_train,:],
                                       session_mat[opto_train,:], 1))
            opto_hyp_train_params = lik_model.x
            _,opto_hyp_nloglik = beh_func(opto_hyp_train_params, glm_target[opto_test],
                                          a_hist[opto_test,:], R_hist[opto_test,:],
                                          uR_hist[opto_test,:], session_mat[opto_test,:],0)
        # Fit on ctrl trials
        if fit_method=='basinhopping':
            minimizer_kwargs = {"method":'L-BFGS-B', "bounds":bounds,
                                "args": (glm_target[ctrl_train], a_hist[ctrl_train,:],
                                         R_hist[ctrl_train,:], uR_hist[ctrl_train,:], 
                                         session_mat[ctrl_train,:], 1)}
            
            # Fit exponential
            beh_func = logit_exp_r_w
            with np.errstate(divide='ignore',invalid='ignore'):
                lik_model = basinhopping(beh_func, initial_params, minimizer_kwargs=minimizer_kwargs, niter=10)
            ctrl_exp_train_params = lik_model.x
            _,ctrl_exp_nloglik = beh_func(ctrl_exp_train_params, glm_target[ctrl_test],
                                          a_hist[ctrl_test,:], R_hist[ctrl_test,:],
                                          uR_hist[ctrl_test,:], session_mat[ctrl_test,:],0)
            
            # Fit hyperbolic
            beh_func = logit_hyp_r_w
            with np.errstate(divide='ignore',invalid='ignore'):
                lik_model = basinhopping(beh_func, initial_params, minimizer_kwargs=minimizer_kwargs, niter=10)
            ctrl_hyp_train_params = lik_model.x
            _,ctrl_hyp_nloglik = beh_func(ctrl_hyp_train_params, glm_target[ctrl_test],
                                          a_hist[ctrl_test,:], R_hist[ctrl_test,:],
                                          uR_hist[ctrl_test,:], session_mat[ctrl_test,:],0)
        else:
            # Fit exponential
            beh_func = logit_exp_r_w
            lik_model = minimize(beh_func, initial_params, bounds=bounds, method=fit_method,
                                 args=(glm_target[ctrl_train], a_hist[ctrl_train,:],
                                       R_hist[ctrl_train,:], uR_hist[ctrl_train,:],
                                       session_mat[ctrl_train,:], 1))
            ctrl_exp_train_params = lik_model.x
            _,ctrl_exp_nloglik = beh_func(ctrl_exp_train_params, glm_target[ctrl_test],
                                          a_hist[ctrl_test,:], R_hist[ctrl_test,:],
                                          uR_hist[ctrl_test,:], session_mat[ctrl_test,:],0)
            
            # Fit hyperbolic
            beh_func = logit_hyp_r_w
            lik_model = minimize(beh_func, initial_params, bounds=bounds, method=fit_method,
                                 args=(glm_target[ctrl_train], a_hist[ctrl_train,:],
                                       R_hist[ctrl_train,:], uR_hist[ctrl_train,:],
                                       session_mat[ctrl_train,:], 1))
            ctrl_hyp_train_params = lik_model.x
            _,ctrl_hyp_nloglik = beh_func(ctrl_hyp_train_params, glm_target[ctrl_test],
                                          a_hist[ctrl_test,:], R_hist[ctrl_test,:],
                                          uR_hist[ctrl_test,:], session_mat[ctrl_test,:],0)
       
        opto_delta_loglik = (-opto_hyp_nloglik) - (-opto_exp_nloglik)
        ctrl_delta_loglik = (-ctrl_hyp_nloglik) - (-ctrl_exp_nloglik)
        
        if nn==0:
            opto_exp_param_mat = opto_exp_train_params.copy()
            ctrl_exp_param_mat = ctrl_exp_train_params.copy()
            opto_hyp_param_mat = opto_hyp_train_params.copy()
            ctrl_hyp_param_mat = ctrl_hyp_train_params.copy()
            opto_dll_mat = opto_delta_loglik
            ctrl_dll_mat = ctrl_delta_loglik
        else:
            opto_exp_param_mat = np.vstack((opto_exp_param_mat, opto_exp_train_params))
            ctrl_exp_param_mat = np.vstack((ctrl_exp_param_mat, ctrl_exp_train_params))
            opto_hyp_param_mat = np.vstack((opto_hyp_param_mat, opto_hyp_train_params))
            ctrl_hyp_param_mat = np.vstack((ctrl_hyp_param_mat, ctrl_hyp_train_params))
            opto_dll_mat = np.vstack((opto_dll_mat, opto_delta_loglik))
            ctrl_dll_mat = np.vstack((ctrl_dll_mat, ctrl_delta_loglik))
            
    fit_df = pd.DataFrame({'n_iter': n_iterations, 'mdl_hist': n_back, 'fit_method': fit_method,
                           'opt_catch': opt_catch,
                           'median_opto_dll': np.nanmedian(opto_dll_mat,axis=0),
                           'median_ctrl_dll': np.nanmedian(ctrl_dll_mat,axis=0),
                           'mean_opto_dll': np.nanmean(opto_dll_mat,axis=0),
                           'mean_ctrl_dll': np.nanmean(ctrl_dll_mat,axis=0),
                           'median_opto_dll_trial': np.nanmedian(opto_dll_mat,axis=0)/(n_per_test),
                           'median_ctrl_dll_trial': np.nanmedian(ctrl_dll_mat,axis=0)/(n_per_test),
                           'median_opto_exp_params': [np.nanmedian(opto_exp_param_mat,axis=0)],
                           'median_ctrl_exp_params': [np.nanmedian(ctrl_exp_param_mat,axis=0)],
                           'median_opto_hyp_params': [np.nanmedian(opto_hyp_param_mat,axis=0)],
                           'median_ctrl_hyp_params': [np.nanmedian(ctrl_hyp_param_mat,axis=0)],
                          })

    return fit_df

#### across-area modelfit iterable

In [18]:
def fit_subsample_given_area(beh_df, area, m_all, fit_method, n_iterations, n_back,opt_catch):
    fit_df = pd.DataFrame([])
    # For given area, estimate all sessions for mouse
    for mm in m_all:
        temp_beh = beh_df.loc[(beh_df['Mouse']==mm) & 
                      ((beh_df['Area']==area) | (beh_df['Area']=='Control'))]
        temp_df = fit_subsample_behlogit(temp_beh, area, fit_method, n_iterations, n_back,opt_catch)
        temp_df['Mouse'] = mm
        temp_df['Area'] = area
        
        fit_df = fit_df.append(temp_df)
    return fit_df

### Load data

In [118]:
# Load session summary
chr2_session_summary = pd.read_pickle(pjoin(opto_dir,'ChR2_session_summary.pkl'))

beh_df = chr2_session_summary.copy()

In [16]:
# Mouse IDs
rsc_m_id = beh_df.loc[(beh_df['Area']=='RSCb'),'Mouse'].unique()
ppc_m_id = beh_df.loc[(beh_df['Area']=='PPCb'),'Mouse'].unique()
m2_m_id = beh_df.loc[(beh_df['Area']=='M2b'),'Mouse'].unique()
loh_m_id = beh_df.loc[(beh_df['Area']=='Control'),'Mouse'].unique()

# Group intersection
loh_rsc_m_id = np.intersect1d(loh_m_id, rsc_m_id)
# rsc_to_use = np.isin(rsc_m_id,loh_rsc_m_id)
loh_to_use = np.isin(loh_m_id, loh_rsc_m_id)
loh_rsc_m_id = loh_m_id[loh_to_use]

loh_ppc_m_id = np.intersect1d(loh_m_id, ppc_m_id)
# ppc_to_use = np.isin(ppc_m_id,loh_ppc_m_id)
loh_to_use = np.isin(loh_m_id, loh_ppc_m_id)
loh_ppc_m_id = loh_m_id[loh_to_use]

loh_m2_m_id = np.intersect1d(loh_m_id, m2_m_id)
loh_to_use = np.isin(loh_m_id, loh_m2_m_id)
loh_m2_m_id = loh_m_id[loh_to_use]

print(loh_ppc_m_id)

['BD032' 'BD033' 'BD109' 'BD110' 'BD111' 'BD112' 'BD115' 'BD116' 'BD118']


### Iterate across areas 

In [48]:
temp_df = fit_subsample_given_area(beh_df, 'RSCb', loh_rsc_m_id, 'basinhopping', 30, 10)
behlogit_out = temp_df.copy()
temp_df = fit_subsample_given_area(beh_df, 'PPCb', loh_ppc_m_id, 'basinhopping',30, 10)
behlogit_out = behlogit_out.append(temp_df)
temp_df = fit_subsample_given_area(beh_df, 'M2b', loh_m2_m_id, 'basinhopping',30, 10)
behlogit_out = behlogit_out.append(temp_df)
temp_df = fit_subsample_given_area(beh_df, 'RSCb', loh_rsc_m_id, 'basinhopping', 30, 10)
behlogit_out = behlogit_out.append(temp_df)
temp_df = fit_subsample_given_area(beh_df, 'PPCb', loh_ppc_m_id, 'basinhopping',30, 10)
behlogit_out = behlogit_out.append(temp_df)
temp_df = fit_subsample_given_area(beh_df, 'M2b', loh_m2_m_id, 'basinhopping',30, 10)
behlogit_out = behlogit_out.append(temp_df)

temp_df = fit_subsample_given_area(beh_df, 'RSCb', loh_rsc_m_id, 'Nelder-Mead', 30, 10)
behlogit_out = behlogit_out.append(temp_df)
temp_df = fit_subsample_given_area(beh_df, 'PPCb', loh_ppc_m_id, 'Nelder-Mead',30, 10)
behlogit_out = behlogit_out.append(temp_df)
temp_df = fit_subsample_given_area(beh_df, 'M2b', loh_m2_m_id, 'Nelder-Mead',30, 10)
behlogit_out = behlogit_out.append(temp_df)

local_dir = 'C://jupyter_notebooks//processed_data'
pickle.dump(behlogit_out,open(pjoin(opto_dir, 'opto_behlogit_RA_10hist.pkl'),'wb'))

292 199 179
174 68 61
66 63 56
143 194 128
163 103 92
142 124 111
227 427 204
92 241 82
101 333 90
310 199 179
147 68 61
176 194 158
295 103 92
90 124 81
121 427 108
156 241 140
96 333 86
151 179 135
341 199 179
235 68 61
85 63 56
99 103 89
107 427 96
57 57 51
187 241 168
89 333 80
264 179 161
